**fastText** - библиотека Facebook для получения векторных представлений (эмбеддингов) слов и классификации текстов с помощью unsupervised и supervised методов машинного обучения. Помимо инструментария создатели библиотеки предоставляют в открытое пользование предобученные модели для различных языков (294 шт.), а также свою мультиязыковую модель. Для нашего исследования мы будем использовать предобученную векторную модель для русского языка.

В качестве основной библиотеки для оценки семантической близости между заданным вопросом и вопросом из нашего датасета мы используем пакет **spacy** и его метод `.similarity()`, который возвращает число с точкой, определяющее насколько каждая пара вопросов схожа по своей семантике. Затем выводим топ-5 кандидатов с наибольшими показателями сходства.

Метод `.similarity()` использует косинусную меру сходства для рассчета близости векторов - это мера сходства между двумя векторами предгильбертового пространства, которая используется для измерения косинуса угла между ними, а косинус угла между двумя векторами — это их скалярное произведение, деленное на длину каждого из двух векторов.

$$\cos ({\bf A},{\bf B})= {{\bf A}\cdot{\bf B} \over \|{\bf A}\|\cdot \|{\bf B}\|} = \frac{ \sum_{i=1}^{n}{{\bf A}_i{\bf B}_i} }{ \sqrt{\sum_{i=1}^{n}{({\bf A}_i)^2}} \sqrt{\sum_{i=1}^{n}{({\bf B}_i)^2}} }$$

где A и B - векторы признаков.

#### Как делали создатели fastText: описание идеи (general model)

Модель, использованная для создания предобученных моделей fastText, происходит из continuous skipgram модели с отрицательным семплированием, описанной  у  Mikolov et al. (2013). Идея основана на дистрибутивной гипотезе,т.е. векторные представления слов нужны, чтобы предсказывать слова, которые находятся в контексте конкретного слова. Таким образом, имея словарь размера $W$, где слову соответствует индекс $w~\in~\{1, ..., W\}$, цель состоит в получении векторного представления для каждого слова $w$. Другими словами, при наличии обучающего корпуса в виде последовательности слов $w_1, ..., w_T$ цель этой skipgram модели состоит в максимизации логарифмической функции правдоподобия (*maximize the following log-likelihood*):

$$\begin{equation*}
  \sum_{t=1}^T \ \sum_{c \in \mathcal{C}_t} \ \log p(w_c \ | \ w_t),
\end{equation*}$$

где контекст $\mathcal{C}_t$ представляет собой множество индексов слов, окружающих слово $w_t$.
Вероятность наблюдения контекстного слова $w_c$ для слова $w_t$ параметризуются с помощью вышеуказанных векторных представлений. Пусть $s$ - функция подсчета (scoring function), которая мапит пары (слово, контекст) со значениями из множества действительных чисел $\mathbb{R}$.  
Авторы не стали использовать softmax для определения контекстных слов, поскольку он предполагает прогнозирование только одного контекстного слова $w_c$ для слова $w_t$. Можно сформулировать проблему предсказания контекстных слов как набор независимых задач бинарной классификации. В таком случае, цель состоит в независимом предсказании наличия (отсутствия) контекстных слов.  
Для слова в позиции $t$ все контекстные слова рассматриваются как положительные примеры, а отрицательные набираются рандомно из словаря. Для выбранного контекста $c$ используют бинарную логистическую функцию потерь, чтобы получить следующую negative log-likelihood:

$$\begin{equation*}
  \log \left(1 + e^{-s(w_t,\ w_c)} \right) + \sum_{n \in \mathcal{N}_{t, c}} \log \left(1 + e^{s(w_t,\ n)}\right),
\end{equation*}$$

где $ \mathcal {N} _ {t, c} $ - набор отрицательных примеров, отобранных из словаря.
Если обозначить логистическую функцию потерь $\ell: x \mapsto \log(1 + e^{-x})$, получится вот так:

\begin{equation*}
\sum_{t=1}^{T}  \left [ \sum_{c \in \mathcal{C}_t} \ell(s(w_t,\ w_c)) + \sum_{n \in \mathcal{N}_{t,c}} \ell(-s(w_t,\ n)) \right ].
\end{equation*}

Естественной параметризацией для функции подсчета $s$ между словом $w_t$ и контекстным словом $w_c$ является использование векторов слов.
Для каждого слова $w$ в словаре определяется два вектора $u_w$ и $v_w$ в пространстве $\mathbb{R}^d$, эти векторы еще называют *input* и *output*. В частности, имеются векторы $\mathbf{u}_{w_t}$ и $\mathbf{v}_{w_c}$, соответствующие словам $w_t$ и $w_c$. Затем score считается как скалярное произведение между векторами слов и контекстов, т.е. как 
$s(w_t, w_c) = \mathbf{u}_{w_t}^{\top} \mathbf{v}_{w_c}$.  

Однако, используя отдельные векторные представления для каждого слова, эта модель игнорирует внутреннюю структуру слова, поэтому авторы придумали, как использовать эту информацию для модификации функции подсчета.
Каждое слово $w$ представлено как мешок символьных $n$-грам. Авторы добавили специальные символы $\texttt{<}$ и $\texttt{>}$ для обозначения начала и конца слова, позволяющие отличить префиксы и суффиксы от других последовательностей символов. Также они добавили само слово $w$ к его $n$-граммам, чтобы получить репрезентацию всего слова. Например, для слова $where$ и $n=3$ получится такая картина: 
$<wh, whe, her, ere, re>$ и само слово $<where>$. Таким образом можно отличать короткие слова от входящих в какое-то слово $n$-грам. На практике они извлекали все $n$-граммы для $n$ \>=3 и <=6.  

Предполагается, что есть словарь $n$-грам размера $G$. Имея слово $w$, набор $n$-грам, представленных в нем определяется как $\mathcal{G}_w \subset \{1, \dots, G \}$. Сопоставляется векторное представление $\mathbf{z}_g$ каждой  $n$-грамме $g$. Слово представляется как сумма векторов его $n$-grams. Таким образом, получаем другую фукцию подсчета:  
$$\begin{equation*}
s(w, c) = \sum_{g \in \mathcal{G}_w} \mathbf{z}_g^\top \mathbf{v}_c.
\end{equation*}$$
Эта простая модель позволяет получать представления между словами, а также хорошо работает для редких слов.  
Для того, чтобы модель была ограничена в своих требованиях к объему памяти, авторы придумали использовать функцию хэширования, которая мапит $n$-граммы в целые числа от 1 до $K$. Они хэшат символьные последовательности с помощью простой хэш-функции *Fowler-Noll-Vo(FNV)*, а именно, ее вариации *FNV-1a*. Значение $K = 2.10^6$. В конечном счете слово представлено его индексом в словаре и множеством хэшированных $n$-грамм, которые он содержит.  
 
 **Датасет**  
 
 Модель тренировалась на дампах википедии, выкаченных для 9 языков: арабского, чешского, немецкого, английского, испанского, французского, русского, итальянского и румынского. Строки были нормализированы с помощью [скрипта на perl](http://mattmahoney.net/dc/textdata "Matt Mahoney’s pre-processing perl script"). Все датасеты перемешивались, модель прогоняли 5 раз.  
 
 **Детали**
 
 Размерность векторного пространства - 300. Для каждого положительного примера собирались 5 рандомных отрицательных с вероятностью пропорциональной квадратному корню из частоты униграммы. Использовалось окно размером $c$, оно выбиралось равномерно от $1$ и $5$ для слов. Для того, чтобы сделать выборку подслов наиболее частотных слов (униграмм), они использовали порог размеров $10^{-4}$. При создании словаря оставлялись только те слова, которые появились в обучающей выборке минимум 5 раз. На английском датасете модель с $n$-граммами тренировалась в $1.5$ раза медленнее, чем только на словах. Были обработаны $105$k слов/подслов/последовательностей.
 Модель имплементирована на C++.
 
 **References**
 
 1. Tomáš Mikolov, Ilya Sutskever, Kai Chen, Greg S. Corrado, and Jeff Dean. 2013. Distributed representations of words and phrases and their compositionality. In Adv. NIPS.
 2. Piotr Bojanowski and Edouard Grave and Armand Joulin and Tomas Mikolov.2016. Enriching Word Vectors with Subword Information.CoRR.